In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import drive 
# /content/drive/My Drive/cs1951a/Lab 3/sonnet1.txt"
from google.colab import drive
drive.mount('/content/drive/')
crop_df = pd.read_csv('/content/drive/My Drive/Data Science: Food Security/data/FAOSTAT_data_en_3-19-2023.csv', encoding='latin-1')

# Change the "Area" column name to "Country"
crop_df = crop_df.rename(columns={"Area":"Country"})

# Select the following columns: Domain, Country (Area), Element, Item, Year, Unit, Value
crop_df = crop_df[["Domain", "Country", "Element", "Item", "Year", "Unit", "Value"]]

# Change all the "Maize (corn)" to just "Maize" (for the Item column)
crop_df = crop_df.replace({"Item": "Maize (corn)"}, {"Item": "Maize"})

# convert all values to t/ha (tonne per hectare)
# ha, hg/ha, tonnes
# Convert yield to t/ha
# crop_df = crop_df.rename(columns={"Element": "Yield"})
crop_df["Unit"] = np.where(crop_df["Element"] == "Yield", "t/ha", crop_df["Unit"])
crop_df["Value"] = np.where(crop_df["Element"] == "Yield", crop_df["Value"] * 0.0001, crop_df["Value"])

# convert "tonnes" to "t"
crop_df = crop_df.replace({"Unit": "tonnes"}, {"Unit": "t"})

crop_df_grouped = crop_df.groupby("Element")
yield_group_df = crop_df_grouped.get_group("Yield")
area_harvest_group_df = crop_df_grouped.get_group("Area harvested")
production_group_df = crop_df_grouped.get_group("Production")

# edit the yield_group_df
yield_group_df = yield_group_df[["Domain", "Country", "Item", "Year", "Value"]]
yield_group_df = yield_group_df.rename(columns={"Value": "Yield", "Item": "Crop"})

# edit area harvest df
area_harvest_group_df = area_harvest_group_df[["Domain", "Country", "Item", "Year", "Value"]]
area_harvest_group_df = area_harvest_group_df.rename(columns = {"Value": "Area harvested", "Item": "Crop"})

# edit production group df
production_group_df = production_group_df[["Domain", "Country", "Item", "Year", "Value"]]
production_group_df = production_group_df.rename(columns = {"Value": "Production", "Item": "Crop"})


# merge the two datasets
merged_df = pd.merge(area_harvest_group_df, production_group_df, on=["Domain", "Country", "Crop", "Year"])

# get rid of rows with 0
merged_df = merged_df[merged_df['Production'] != 0]
merged_df = merged_df[merged_df['Area harvested'] != 0]

# calculate production per area (yield)
merged_df["Production/Area"] = merged_df["Production"] / merged_df["Area harvested"]

# merge the merged_df with the yield df
all_crop_df = pd.merge(yield_group_df, merged_df, on=["Domain", "Country", "Crop", "Year"])
all_crop_df = all_crop_df.drop(columns=["Domain", "Yield"])
all_crop_df = all_crop_df.rename(columns = {"Production/Area": "Yield"})

all_crop_df.head()
all_crop_df.to_csv('/content/drive/My Drive/Data Science: Food Security/data/FAOSTAT_data_en_3-19-2023_preprocessed.csv')


# Get range of values for Data Spec
min_val_yield = all_crop_df["Yield"].min()
max_val_yield = all_crop_df["Yield"].max()
print("Yield Minimum value:", min_val_yield)
print("Yield Maximum value:", max_val_yield)

min_val_ah = all_crop_df["Area harvested"].min()
max_val_ah = all_crop_df["Area harvested"].max()
print("Area harvested Minimum value:", min_val_ah)
print("Area harvested Maximum value:", max_val_ah)

min_val_prod = all_crop_df["Production"].min()
max_val_prod = all_crop_df["Production"].max()
print("Production Minimum value:", min_val_prod)
print("Production Maximum value:", max_val_prod)

# Get the total number of rows in a Pandas dataframe
num_rows = all_crop_df.shape[0]

print("Total number of rows:", num_rows)

# separate into Wheat, Maize, Rice columns
# all_items_grouped = all_items_df.groupby("Item")
# maize_group_df = all_items_grouped.get_group("Maize")
# rice_group_df = all_items_grouped.get_group("Rice")
# wheat_group_df = all_items_grouped.get_group("Wheat")

# # reformat maize group df
# maize_group_df = maize_group_df[["Domain", "Country", "Year", "Yield", "Area harvested", "Production", "Production/Area"]]
# maize_group_df = maize_group_df.rename(columns = {"Yield": "Maize_Yield", "Area harvested": "Maize_Area_Harvested", "Production": "Maize_Production", "Production/Area": "Maize_Production/Area"})

# # reformat rice group df
# rice_group_df = rice_group_df[["Domain", "Country", "Year", "Yield", "Area harvested", "Production", "Production/Area"]]
# rice_group_df = rice_group_df.rename(columns = {"Yield": "Rice_Yield", "Area harvested": "Rice_Area_Harvested", "Production": "Rice_Production", "Production/Area": "Rice_Production/Area"})

# # reformat wheat group df
# wheat_group_df = wheat_group_df[["Domain", "Country", "Year", "Yield", "Area harvested", "Production", "Production/Area"]]
# wheat_group_df = wheat_group_df.rename(columns = {"Yield": "Wheat_Yield", "Area harvested": "Wheat_Area_Harvested", "Production": "Wheat_Production", "Production/Area": "Wheat_Production/Area"})

# # join maize, rice and wheat group df
# final_df = pd.merge(pd.merge(maize_group_df, rice_group_df, on=["Domain", "Country", "Year"]), wheat_group_df, on=["Domain", "Country", "Year"])
# final_df = final_df.drop(columns=["Domain"])
# final_df.head()
# all_items_df.head()

# PROJECT IMPACT DF 
# projected_impact_df = pd.read_excel('/content/drive/My Drive/Data Science: Food Security/data/Projected_impacts_datasheet_11.24.2021.xlsx')

# # drop non-relevant columns
# projected_impact_df = projected_impact_df.drop(columns=["ID", "Scale", "Site(location)", "Time slice", "Reference", "doi", "Publication year", "Ref No ", " Methods"
# , 'Note1 \n(* = corrected by HW) ', 'Note2 \n(* = Local temperature is  estimated )', 'Note3 \n(* = Local delta Pr is  estimated )', 'Note4 \n(* = Global temperature is  estimated )'])

# # get all the column names (after dropping certain columns):
# all_column_names = list(projected_impact_df.columns.values)

# # replace all missing values with 0
# nan_cols = projected_impact_df.columns[projected_impact_df.isna().any()]
# projected_impact_df = projected_impact_df.replace(np.nan, 0)
# # for column_name in nan_cols:
# #   projected_impact_df[column_name] = projected_impact_df[column_name].astype(float).interpolate()

# projected_impact_df = projected_impact_df.rename(columns = {"Baseline_Mid-point": "Year"})

# # visualize df
# # projected_impact_df.head()

# # Join all_items_df and projected_impact_df
# merged_crop_and_projected_df = pd.merge(all_crop_df, projected_impact_df, on=["Country", "Crop", "Year"])

# # remove duplicate rows (based on all columns)
# merged_crop_and_projected_df.drop_duplicates()
# merged_crop_and_projected_df.head()


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Yield Minimum value: 0.0001261869459604404
Yield Maximum value: 36.76190476190476
Area harvested Minimum value: 1.0
Area harvested Maximum value: 48178000.0
Production Minimum value: 1.0
Production Maximum value: 412262180.0
Total number of rows: 12834
